In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/2022-ml-midterm-p3/submit.csv
/kaggle/input/2022-ml-midterm-p3/train.csv
/kaggle/input/2022-ml-midterm-p3/test.csv


In [2]:
import random
import os
seed = 42
random.seed(seed)
np.random.seed(seed)
os.environ["PYTHONHASHSEED"] = str(seed)

In [3]:
sample = pd.read_csv('/kaggle/input/2022-ml-midterm-p3/submit.csv')
train = pd.read_csv('/kaggle/input/2022-ml-midterm-p3/train.csv')
test = pd.read_csv('/kaggle/input/2022-ml-midterm-p3/test.csv')

데이터를 불러오기 위해서 pd.read_csv() 함수를 사용하였다.

In [4]:
train

,index,a,b,c,d,e,f,g,h,i,j,k,l,m,price
0,0,0.14150,0.0,6.91,0,0.448,6.169,6.6,5.7209,3,233,17.9,383.37,5.81,25.3
1,1,0.15445,25.0,5.13,0,0.453,6.145,29.2,7.8148,8,284,19.7,390.68,6.86,23.3
2,2,16.81180,0.0,18.10,0,0.700,5.277,98.1,1.4261,24,666,20.2,396.90,30.81,7.2
3,3,0.05646,0.0,12.83,0,0.437,6.232,53.7,5.0141,5,398,18.7,386.40,12.34,21.2
4,4,8.79212,0.0,18.10,0,0.584,5.565,70.6,2.0635,24,666,20.2,3.65,17.16,11.7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
399,399,0.03548,80.0,3.64,0,0.392,5.876,19.1,9.2203,1,315,16.4,395.18,9.25,20.9
400,400,0.09164,0.0,10.81,0,0.413,6.065,7.8,5.2873,4,305,19.2,390.91,5.52,22.8
401,401,5.87205,0.0,18.10,0,0.693,6.405,96.0,1.6768,24,666,20.2,396.90,19.37,12.5
402,402,0.33045,0.0,6.20,0,0.507,6.086,61.5,3.6519,8,307,17.4,376.75,10.88,24.0


In [5]:
X_train = train.drop(['index','price'],axis=1)
y_train = train['price']
X_test = test.drop(['index'],axis=1)

데이터를 살펴보니 'index'가 공통적으로 훈련데이터와 학습데이터에 들어있어서 제거해주었고, 훈련데이터에서는 학습을 하기위해서 'price'라는 라벨을 따로 떼서 다른 변수에 담아주었다.

In [6]:
X_train

,a,b,c,d,e,f,g,h,i,j,k,l,m
0,0.14150,0.0,6.91,0,0.448,6.169,6.6,5.7209,3,233,17.9,383.37,5.81
1,0.15445,25.0,5.13,0,0.453,6.145,29.2,7.8148,8,284,19.7,390.68,6.86
2,16.81180,0.0,18.10,0,0.700,5.277,98.1,1.4261,24,666,20.2,396.90,30.81
3,0.05646,0.0,12.83,0,0.437,6.232,53.7,5.0141,5,398,18.7,386.40,12.34
4,8.79212,0.0,18.10,0,0.584,5.565,70.6,2.0635,24,666,20.2,3.65,17.16
...,...,...,...,...,...,...,...,...,...,...,...,...,...
399,0.03548,80.0,3.64,0,0.392,5.876,19.1,9.2203,1,315,16.4,395.18,9.25
400,0.09164,0.0,10.81,0,0.413,6.065,7.8,5.2873,4,305,19.2,390.91,5.52
401,5.87205,0.0,18.10,0,0.693,6.405,96.0,1.6768,24,666,20.2,396.90,19.37
402,0.33045,0.0,6.20,0,0.507,6.086,61.5,3.6519,8,307,17.4,376.75,10.88


In [7]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train_std = sc.fit_transform(X_train)
X_test_std = sc.transform(X_test)

모델의 성능을 더욱 높이고자 StandardScaler을 통해서 스케일 정규화를 진행해주었다.

In [8]:
from sklearn.neighbors import KNeighborsRegressor
model = KNeighborsRegressor().fit(X_train_std,y_train)
y_test = model.predict(X_test_std)

해당 문제는 땅값을 예측하는 문제로 회귀 문제에 해당하기에 먼저 회귀모델을 만들어야한다고 생각했고 학습데이터의 수가 400개 정도로 많으므로 KNeighborsRegressor을 선언하고 학습을 시켰다.

In [9]:
#sample['price'] = y_test
#sample.to_csv('submit.csv',index=False)

모델을 통해서 에측한 값을 csv에 저장하기 위해서 .to_csv라는 함수를 사용했다.

In [10]:
from sklearn.ensemble import RandomForestRegressor
model1 = RandomForestRegressor().fit(X_train_std,y_train)
y_test1 = model1.predict(X_test_std)

해당 문제는 회귀 문제로 회귀모델을 만들어야겠다고 생각했고, 학습의 데이터의 수가 많고 설명보다는 예측을 필요로 하는 문제이기에 DecisionTreeRegressor보다는 RandomForestRegressor가 더 적합하다고 생각하여 RandomForestRegressor을 선언하고 학습을 시켰다.

In [11]:
sample['price'] = y_test1
sample.to_csv('submit1.csv',index=False)

모델을 통해서 에측한 값을 csv에 저장하기 위해서 .to_csv라는 함수를 사용했다.

KNeighborsRegressor으로 모델을 만들고 파라미터를 default 값으로 설정후 캐글에 제출해보니 2.91705 가 나왔고 RandomForestRegressor로 모델을 만들고 파라미터를 default 값으로 설정후 캐글에 제출해보니 2.26287 가 나왔다. 이를 통해서 분석을 해보자면 해당 데이터에서는 우선 학습데이터가 400개로 많다면 많지만 어찌보면 그렇게 많다고도 생각할 수 없기에 RandomForestRegressor가가 더 성능이 좋을 것으로 예측했다. 이에 캐글의 점수도 RandomForestRegressor의 성능이 더 좋다고 나왔다. 사실 점수로 따져보면 0.7점 정도 차이가 나서 비슷한 성능을 보여준다고도 생각할 수 있지만 KNeighborsRegressor의 단점인 최적의 k값과 거리척도 방식을 사용자가 직접 구해야한다는 점에서 성능이 다소 낮게 나온 것으로 생각한다.